In [1]:
from openpyxl import load_workbook
from snownlp import SnowNLP



In [2]:


# 文件路径
file_path = '/Users/ccmac/Desktop/2022年测.xlsx'

try:
    # 加载Excel工作簿
    wb = load_workbook(file_path)
    ws = wb.active

    # 检查是否已有“情感得分”列
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            if cell.value == '情感得分':
                sentiment_score_col = cell.column
                break
    else:  # 如果没有找到情感得分列
        sentiment_score_col = ws.max_column + 1
        ws.cell(row=1, column=sentiment_score_col, value='情感得分')

    # 找到最后一行已处理的数据
    last_processed_row = 1
    for row in range(2, ws.max_row + 1):
        if ws.cell(row=row, column=sentiment_score_col).value is None:
            last_processed_row = row
            break

    # 从断点开始处理数据
    for row in range(last_processed_row, ws.max_row + 1):
        post_content = ws.cell(row=row, column=1).value  # 假设帖子内容在第一列
        if post_content and isinstance(post_content, str):
            sentiment_score = SnowNLP(post_content).sentiments
            ws.cell(row=row, column=sentiment_score_col, value=sentiment_score)

    # 保存工作簿
    wb.save(file_path)

except Exception as e:
    print(f"发生错误：{e}")
finally:
    # 确保工作簿被关闭
    if 'wb' in locals() and wb is not None:
        wb.close()